In [27]:
import boto3
boto_client = boto3.setup_default_session(region_name='eu-west-1')

In [28]:
def create_ForYou_table(dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb')

    table = dynamodb.create_table(
        TableName='ForYou',
        KeySchema=[
            {
                'AttributeName': 'user_id',
                'KeyType': 'HASH'  # Partition key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'user_id',
                'AttributeType': 'S'
            },

        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        }
    )
    return table

In [29]:
ForYou_table = create_ForYou_table()

In [32]:
def put_recs(user_id, recs_list, type_list, score_list, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb')

    table = dynamodb.Table('ForYou')
    response = table.put_item(
       Item={
           'user_id': user_id,
           'item_id': recs_list,
           'item_type' : type_list,
           'item_score' : score_list
        }
    )
    return response

In [33]:
recs_resp = put_recs(
    'BBEU00021416',
    ['movida_52859'] * 20 ,
    ['BRAND'] * 20,
    list(reversed(range(20)))
)

In [5]:
from botocore.exceptions import ClientError

In [34]:
def get_recs(user_id, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb')

    table = dynamodb.Table('ForYou')

    try:
        response = table.get_item(Key={'user_id': user_id})
    except ClientError as e:
        print(e.response['Error']['Message'])
    else:
        return response['Item']

In [35]:
rec = get_recs('BBEU00021416')
if rec:
    print("Get rec succeeded:")
    print(rec)

Get rec succeeded:
{'user_id': 'BBEU00021416', 'item_score': [Decimal('0.975629627704620361328125'), Decimal('0.971547305583953857421875'), Decimal('0.963755428791046142578125'), Decimal('0.955491244792938232421875'), Decimal('0.953596591949462890625'), Decimal('0.9420344829559326171875'), Decimal('0.932163894176483154296875'), Decimal('0.92504155635833740234375'), Decimal('0.915787756443023681640625'), Decimal('0.91529560089111328125'), Decimal('0.914229333400726318359375'), Decimal('0.9124252796173095703125'), Decimal('0.9094946384429931640625'), Decimal('0.904664814472198486328125'), Decimal('0.9028618335723876953125'), Decimal('0.902654230594635009765625'), Decimal('0.89556944370269775390625'), Decimal('0.891643464565277099609375'), Decimal('0.890914976596832275390625'), Decimal('0.88853418827056884765625')], 'item_id': ['BBJ956271HVOD', 'BBJ956277HVOD', 'BBJ956273HVOD', 'BBJ956275HVOD', 'BBJ956287HVOD', 'BBJ374920HVOD', 'BBJ1621565A', 'BBJ269926HVOD', 'movida_10027073', 'BBJ236657

In [41]:
list(zip(rec['item_type'],rec['item_id']))

[('PROGRAM', 'BBJ956271HVOD'),
 ('PROGRAM', 'BBJ956277HVOD'),
 ('PROGRAM', 'BBJ956273HVOD'),
 ('PROGRAM', 'BBJ956275HVOD'),
 ('PROGRAM', 'BBJ956287HVOD'),
 ('PROGRAM', 'BBJ374920HVOD'),
 ('PROGRAM', 'BBJ1621565A'),
 ('PROGRAM', 'BBJ269926HVOD'),
 ('COLLECTION', 'movida_10027073'),
 ('PROGRAM', 'BBJ2366579A'),
 ('PROGRAM', 'BBJ316698HVOD'),
 ('PROGRAM', 'BBJ1769219A'),
 ('PROGRAM', 'BBJ1714884A'),
 ('PROGRAM', 'BBJ269878HVOD'),
 ('PROGRAM', 'BBJ374923HVOD'),
 ('PROGRAM', 'BBJ956294HVOD'),
 ('PROGRAM', 'BBJ1620056A'),
 ('PROGRAM', 'BBJ2417987A'),
 ('PROGRAM', 'BBJ1708699A'),
 ('PROGRAM', 'BBJ374599HVOD')]

In [40]:
rec['item_type']

['PROGRAM',
 'PROGRAM',
 'PROGRAM',
 'PROGRAM',
 'PROGRAM',
 'PROGRAM',
 'PROGRAM',
 'PROGRAM',
 'COLLECTION',
 'PROGRAM',
 'PROGRAM',
 'PROGRAM',
 'PROGRAM',
 'PROGRAM',
 'PROGRAM',
 'PROGRAM',
 'PROGRAM',
 'PROGRAM',
 'PROGRAM',
 'PROGRAM']

In [39]:
rec['item_id']

['BBJ956271HVOD',
 'BBJ956277HVOD',
 'BBJ956273HVOD',
 'BBJ956275HVOD',
 'BBJ956287HVOD',
 'BBJ374920HVOD',
 'BBJ1621565A',
 'BBJ269926HVOD',
 'movida_10027073',
 'BBJ2366579A',
 'BBJ316698HVOD',
 'BBJ1769219A',
 'BBJ1714884A',
 'BBJ269878HVOD',
 'BBJ374923HVOD',
 'BBJ956294HVOD',
 'BBJ1620056A',
 'BBJ2417987A',
 'BBJ1708699A',
 'BBJ374599HVOD']

In [23]:
from decimal import Decimal

In [31]:
def update_recs(user_id, recs_list, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb')

    table = dynamodb.Table('ForYou')

    response = table.update_item(
        Key={
            'user_id': user_id,
            },
        UpdateExpression="set recs=:r",
        ExpressionAttributeValues={
            ':r': recs_list
        },
        ReturnValues="UPDATED_NEW"
    )
    return response

In [33]:
update_response = update_recs('22', ['a', 'b', 'd'])
print("Update recs succeeded:")
print(update_response)
print(update_response['Attributes']['recs'])

Update recs succeeded:
{'Attributes': {'recs': ['a', 'b', 'd']}, 'ResponseMetadata': {'RequestId': 'RHLRPJ0TLIOVIK3GMA5CFO75PRVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Tue, 06 Oct 2020 13:14:50 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '61', 'connection': 'keep-alive', 'x-amzn-requestid': 'RHLRPJ0TLIOVIK3GMA5CFO75PRVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '1678182544'}, 'RetryAttempts': 0}}
['a', 'b', 'd']
